<a href="https://colab.research.google.com/github/tatiholanda/frtensorflow2/blob/main/Projeto_VC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar tensorflow

In [ ]:
!pip install tensorflow


Importação das bibliotecas que serão usadas


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

Carregar e Preparar o Conjunto de Dados MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizando os dados
x_train, x_test = x_train / 255.0, x_test / 255.0

# Redimensionando as imagens para 28x28x1
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

Visualização de Algumas Imagens do Conjunto de Dados

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(y_train[i])
plt.show()


Construção do Modelo de Rede Neural

In [ ]:
model = Sequential([
  Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=(2, 2)),
  Dropout(0.25),
  Flatten(),
  Dense(128, activation='relu'),
  Dropout(0.5),
  Dense(10, activation='softmax')
])

model.compile(optimizer=
              'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Treinamento do Modelo e Armazenamento do Histórico

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Gráficos de Acurácia e Perda

In [ ]:
# Plotar a acurácia de treinamento e validação
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.title('Acurácia por Épocas')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

# Plotar a perda de treinamento e validação
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda de Treinamento')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Perda por Épocas')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.show()


Avaliação do Modelo e Matriz de Confusão

In [ ]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Acurácia: {accuracy*100}%")

# Previsões do modelo
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_test

# Computar a matriz de confusão
cm = confusion_matrix(y_true, y_pred_classes)

# Visualizar a matriz de confusão
plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão')
plt.ylabel('Valores Reais')
plt.xlabel('Valores Previstos')
plt.show()


Visualização dos Filtros da CNN

In [ ]:
def visualize_filters(model):
    filters, biases = model.layers[0].get_weights()
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)

    n_filters = 6  # Número de filtros para visualizar
    n_columns = 3   # Número de colunas na visualização

    plt.figure(figsize=(12, 8))
    for i in range(n_filters):
        f = filters[:, :, :, i]
        for j in range(1):  # Alterado para 1, assumindo 1 canal
            ax = plt.subplot(n_filters, n_columns, i * n_columns + j + 1)
            ax.set_xticks([])
            ax.set_yticks([])
            plt.imshow(f[:, :, j], cmap='gray')
    plt.show()

visualize_filters(model)
